# Experiment Workflows

A `Workflow` contains a set of tasks to be run and supplies the tasks their options and saves their inputs and outputs. 
When run, a `Workflow` function builds a graph of tasks that will be executed later.
This graph may be inspected and extended.
The graph of tasks is not executed directly by Python, but by a workflow engine provided by the library.

In this tutorial we will take a look at using workflows for simple tune-up experiments.

Let's get started.

## Setting up a device and session

Build your LabOne Q `DeviceSetup`, qubits and `Session` as normal. Here we import an example from the applications library's test suite (this will change in the near future):

In [ ]:
import numpy as np
from laboneq.core.exceptions import LabOneQException
from laboneq.simple import *

from laboneq_applications.qpu_types.tunable_transmon import demo_platform

In [ ]:
# Create a demonstration QuantumPlatform for a tunable-transmon QPU:
qt_platform = demo_platform(n_qubits=6)

# The platform contains a setup, which is an ordinary LabOne Q DeviceSetup:
setup = qt_platform.setup

# And a tunable-transmon QPU:
qpu = qt_platform.qpu

# Inside the QPU, we have qubits, which is a list of six LabOne Q Application
# Library TunableTransmonQubit qubits:
qubits = qpu.qubits

In [ ]:
session = Session(setup)
session.connect(do_emulation=True)

## Experiments as Workflows

A `Workflow` is a collection of logically connected `Tasks` whose inputs and outputs depend on each other. We use `Workflows` to implement experiments. Experiment Workflows have a few standard tasks:

- `create_experiment` for creating the experimental pulse sequence
- `compile_experiment` for compiling the `Experiment` returned by `create_experiment`
- `run_experiment` for running the `CompiledExperiment` returned by `compile_experiment`

Let's see what the experiment `Workflow` looks like for the amplitude Rabi.

In [ ]:
from laboneq_applications.experiments import amplitude_rabi
from laboneq_applications.qpu_types.tunable_transmon import TunableTransmonOperations

Inspect the source code of the `amplitude_rabi` `Workflow` to see what tasks it has.

In [ ]:
amplitude_rabi.experiment_workflow.src

### Run the experiment

In [ ]:
qop = TunableTransmonOperations()
amplitudes = np.linspace(0.0, 0.9, 10)
options = amplitude_rabi.experiment_workflow.options()
options.count(10)
options.averaging_mode("cyclic")
rabi_tb = amplitude_rabi.experiment_workflow(
    session,
    qpu,
    qubits[0],
    amplitudes,
    options=options,
)

### Inspect the experiment Workflow

Inspect the input parameters to the `amplitude_rabi` `Workflow`

In [ ]:
rabi_tb.input

Inspect the tasks inside the Workflow

In [ ]:
result = rabi_tb.run()
[t.name for t in result.tasks]

Inspect the source code of `create_experiment` to see how the experiment pulse sequence was created.

In [ ]:
result.tasks["create_experiment"].src

Inspect the LabOne Q Experiment object returned by `create_experiment`

In [ ]:
print(result.tasks["create_experiment"].output)
# Or alternatively:
#    print(result.tasks[0].output)

Inspect the LabOne Q CompiledExperiment object returned by `compile_experiment`

In [ ]:
print(result.tasks["compile_experiment"].output)

In [ ]:
# inspect pulse sequence with plot_simulation
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(
    result.tasks["compile_experiment"].output,
    signal_names_to_show=["drive", "measure"],
    start_time=0,
    length=50e-6,
)

Inspect the acquired results

In [ ]:
acquired_data = result.tasks["run_experiment"].output  # the acquired results
acquired_data

In [ ]:
acquired_data.q0.result

In [ ]:
# inspect pulse sequence with plot_simulation
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(
    result.tasks["compile_experiment"].output,
    signal_names_to_show=["drive", "measure"],
    start_time=0,
    length=50e-6,
)

### Inspect results after an error

If an error occurs during the execution of `amplitude_rabi`, we can inspect the tasks that have run up to the task that produced the error using `recover()`. This is particularly useful to inspect the experiment pulse sequence in case of a compilation or measurement error.

Let's introduce a compilation error by sweeping the ampltude to values larger than 1, which is not allowed.

In [ ]:
qop = TunableTransmonOperations()
amplitudes = np.linspace(0.0, 1.5, 10)
options = amplitude_rabi.experiment_workflow.options()
options.count(10)

# here we catch the exception so that the notebook can keep executing
try:
    rabi_tb = amplitude_rabi.experiment_workflow(
        session,
        qpu,
        qubits[0],
        amplitudes,
        options=options,
    ).run()
except LabOneQException as e:
    print("ERROR: ", e)

In [ ]:
result = amplitude_rabi.experiment_workflow.recover()
result

In [ ]:
# inspect the experiment section tree
print(result.tasks["create_experiment"].output)

### Run experiment workflow using qubits with temporarily modified parameters

It is possible to run an experiment workflow using qubits with temporarily modified parameters. This is useful for testing or debugging purposes.

To do this, we first clone the qubits from the original qubits and modify the parameters of the cloned qubits.
The experiment workflow is then run using the cloned qubits.

Let's run the amplitude Rabi experiment workflow with a new set of qubits with modified parameters.

In [ ]:
temp_qubits = qpu.copy_qubits()
temp_qubits[0].parameters.ge_drive_length = 1000e-9  # 51ns in the original qubits

result_unmodified = amplitude_rabi.experiment_workflow(
    session,
    qpu,
    qubits[0],  # pass original qubits
    np.linspace(0, 1, 21),
    options=options,
).run()

result_modified = amplitude_rabi.experiment_workflow(
    session,
    qpu,
    temp_qubits[0],  # pass temporary qubits
    np.linspace(0, 1, 21),
    options=options,
).run()

In [ ]:
# compare the two pulse sequences
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(
    result_unmodified.tasks["compile_experiment"].output,
    signal_names_to_show=["drive"],
    start_time=0,
    length=5e-6,
)

In [ ]:
plot_simulation(
    result_modified.tasks["compile_experiment"].output,
    signal_names_to_show=["drive", "measure"],
    start_time=0,
    length=5e-6,
)